<h1>VacationPy: An exercise in making API calls to analyze data.</h1>

This code will use Geocoding and API calls to OpenWeather to analyze data from 500 Worldwide cities in order to identify an ideal vacation spot in terms of pleasant weather.

In [1]:
# Dependencies
import requests
import json
from citipy import citipy
import numpy as np
from config3 import owkey
import geoip2.database
import geoip2.webservice
import pandas as pd
import random
import time
import matplotlib.pyplot as plt
from scipy.stats import sem
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
import gmaps
from config2 import gkey
import pprint
import time

In [2]:
#Import CSV file with city data
file="worldcitiespop_reduced.csv"
cities_df=pd.read_csv(file)

In [3]:
#Extract a random sample (without replacement) of 500 cities from the CSV file
simpcities_df = cities_df[["Latitude","Longitude"]]
cities_dict=simpcities_df.set_index("Latitude")["Longitude"].to_dict()
random_500=dict(random.sample(cities_dict.items(), 500))

In [4]:
#Create a dataframe with weather info

weather_df= pd.DataFrame({
    "Name": [], 
    "Temperature (F)": [],
    "Max Temperature (F)":[],
    "Humidity (%)": [], 
    "Cloudiness (%)": [], 
    "Windspeed (mph)": [], 
    
})

#Loop through the dataframe to make API calls and get the weather in 500 cities

call_counter=0
for lat in random_500:
    api_call = "http://api.openweathermap.org/data/2.5/weather?lat=" + str(lat) + "&lon=" + str(random_500[lat]) + "&units=imperial" + "&appid=" + owkey
    api_response = requests.get(api_call)
    api_json = api_response.json()
    name=api_json["name"]
    temp=api_json["main"]["temp"]
    temp_max=api_json["main"]["temp_max"]
    humidity=api_json["main"]["humidity"]
    clouds=api_json["clouds"]["all"]
    speed=api_json["wind"]["speed"]
    
    #append the weather values to the dataframe
    weather_df=weather_df.append({'Name' : name , 
                                  'Temperature (F)' : temp, 
                                  "Humidity (%)": humidity, 
                                  "Cloudiness (%)": clouds, 
                                  "Windspeed (mph)": speed,
                                  "Max Temperature (F)":temp_max,
                                  "Latitude": lat, 
                                  "Longitude": random_500[lat] } , ignore_index=True)
    
    call_counter+=1
    if call_counter==19:
        time.sleep(1)
        call_counter=0
        
        continue

In [5]:
weather_df["Distance From Equator"]=abs(weather_df["Latitude"])
weather_df

,Name,Temperature (F),Max Temperature (F),Humidity (%),Cloudiness (%),Windspeed (mph),Latitude,Longitude,Distance From Equator
0,Hrvatska Kostajnica,66.20,66.20,88.0,75.0,3.36,45.219444,16.608333,45.219444
1,Gongchuan,64.80,64.80,96.0,0.0,3.44,25.960556,117.562222,25.960556
2,Jiangbei,73.15,73.15,77.0,0.0,2.10,26.271811,115.453081,26.271811
3,Kushkak,86.27,86.27,14.0,93.0,18.05,32.359875,61.479147,32.359875
4,Lamin,87.80,87.80,45.0,37.0,12.75,13.445833,-16.380556,13.445833
...,...,...,...,...,...,...,...,...,...
495,Xiachuan,82.33,82.33,85.0,39.0,5.28,21.645961,112.636006,21.645961
496,Iporanga,75.51,75.51,51.0,10.0,3.69,-24.851500,-48.480700,24.851500
497,Kajrān,71.94,71.94,21.0,15.0,1.79,33.122007,65.070082,33.122007
498,Clonas-sur-Varèze,75.22,78.80,41.0,9.0,21.92,45.413264,4.789964,45.413264


In [6]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)
weather_df = weather_df.dropna()
humidity_weight = weather_df["Humidity (%)"].astype(float)

In [7]:

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig 

Figure(layout=FigureLayout(height='420px'))

In [8]:
#Filter the data to find the locatons with the best weather

#----Filter by temperature
newlocs_df=weather_df.loc[weather_df["Max Temperature (F)"]>70]
newlocs_df=newlocs_df.loc[newlocs_df["Max Temperature (F)"]<80]

#----Filter by windspeed
newlocs_df=newlocs_df.loc[newlocs_df["Windspeed (mph)"]<10]


#----Filter by cloudiness
newlocs_df=newlocs_df.loc[newlocs_df["Cloudiness (%)"]==0]


In [9]:
#Build new heat map of the (reduced) list of ideal weather locations

newlocs = newlocs_df[["Latitude", "Longitude"]].astype(float)
humidity_weight2 = newlocs_df["Humidity (%)"].astype(float)

fig2 = gmaps.figure()
heat_layer2 = gmaps.heatmap_layer(newlocs, weights=humidity_weight2, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig2.add_layer(heat_layer2)

fig2

Figure(layout=FigureLayout(height='420px'))

In [10]:
#Use Geocode API to loop through the list of ideal weather locations and get
#info about the nearest hotel within 50,000 meters of each (the homework said 5,000, but this yielded few results).

##NOTE: because data is sampled randomly and Geocode data is incomplete, sometimes missing values are returned---

#----Create new columns in the dataframe to hold new info

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
newlocs_df["City"] = ""
newlocs_df["Country"]=""
newlocs_df["City"]=""
#newlocsindex=newlocs_df.setindex("Name")


#----Get the nearest hotel by latitude and longitude
for index, row in newlocs_df.iterrows():
    
    hotelzone_lat = row['Latitude']
    hotelzone_long=row['Longitude']
    
    params = {
    "location": str(hotelzone_lat)+","+str(hotelzone_long),
    "radius": 50000,
    "input": "",
    "type":"lodging",
    "key": gkey,}


    
    response = requests.get(base_url, params=params).json()
    results = response['results']

#----Try to print the hotel name, also save that to the dataframe

    try:
        print(f"Hotel name is {results[0]['name']}.")
        
        newlocs_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Hotel name not found.")
        
#----Make an new API call to collect country and city     
    newbase_url="https://maps.googleapis.com/maps/api/geocode/json?"
    
    params = {
    "latlng": str(hotelzone_lat)+","+str(hotelzone_long),
    "key": gkey,}
    

#----Try to print the hotel city and country, also save that to the dataframe     
    try:
        response2 = requests.get(newbase_url, params=params).json()
        city= newlocs_df.loc[index, 'City']=response2["results"][0]["address_components"][2]["long_name"]
        newlocs_df.loc[index, 'City'] = city
        print(f"City is {city}.")
    except:
        print("City not found.")
    try:
        country= newlocs_df.loc[index, 'Country']=response2["results"][0]["address_components"][5]["long_name"]
        newlocs_df.loc[index, 'Country'] = country
        print(f"Country is {country}.")
        
    except (KeyError, IndexError):
        print("Country not found.")
        
    print("------------")
    

Hotel name is Ganzhou Xingguo Huanglongshun Hotel.
City is Ganzhou Shi.
Country not found.
------------
Hotel name is Qingcheng Tourism Industry.
City is Qingyuan Shi.
Country not found.
------------
Hotel name is Howard Johnson Plaza Hotel Mayorazgo.
City is Crespo.
Country is Argentina.
------------
Hotel name is Campanile Val de France.
City is Jouy-sur-Morin.
Country is France.
------------
Hotel name is Meliá Bilbao.
City is Gordexola.
Country is Spain.
------------
Hotel name is The Zuiranju Holiday Hotel.
City is Yunfu Shi.
Country not found.
------------
Hotel name is Wuzhou Kailai Hotel.
City is Maoming Shi.
Country not found.
------------
Hotel name is Hengshan Yingbin Mountain Villa Hotel.
City is Hengyang Shi.
Country not found.
------------
Hotel name is وچه خوړه،Wacha Khwara.
City is Khost.
Country not found.
------------
Hotel name is Parador de Santo Estevo.
City is Casanova.
Country is Spain.
------------
Hotel name is Appart'City Pau Centre - Appart Hôtel.
City is Gaz

In [11]:
#Create a column in the dataframe that combines the relevant info so that I can drop it in the infobox

newlocs_df["BoxStuff"]= (newlocs_df["Hotel Name"] + ": "+ newlocs_df["City"]) + ", " +newlocs_df["Country"]

In [12]:
#Make those infoboxes

boxes = newlocs_df["BoxStuff"].tolist()

# Create a map layer that flags the hotels and puts my infoboxes inside them

last_locs = newlocs_df[["Latitude", "Longitude"]].astype(float)

hotel_layer = gmaps.symbol_layer(
    last_locs, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[item for item in boxes])

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))